In [152]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [153]:
from utils import query, job_config, render, register_filter, register_global
import pandas as pd

In [156]:
properties = {
    "decentralized local nesterov momentum with time-varying exponential graph": {
        "algorithm": "gossip",
        "momentum": 0.9,
        "momentum_type": "local",
        "topology": "exponential",
        "lr": "normal",
    },
    "decentralized quasi-global momentum with time-varying exponential graph": {
        "algorithm": "gossip",
        "momentum": 0.9,
        "momentum_type": "quasi-global",
        "topology": "exponential",
        "lr": "normal",
    },
    "decentralized quasi-global momentum with ring topology": {
        "algorithm": "gossip",
        "momentum": 0.9,
        "momentum_type": "quasi-global",
        "topology": "ring",
        "lr": "normal",
    },
    "RelaySum/model (with local nesterov momentum, 3-tree, w/o lr correction)": {
        "algorithm": "RelaySum/Model",
        "momentum": 0.9,
        "lr": "small",
        "momentum_type": "local",
        "topology": "3-tree",
    },
    "RelaySum/model (with local nesterov momentum, strip, w/o lr correction)": {
        "algorithm": "RelaySum/Model",
        "momentum": 0.9,
        "momentum_type": "local",
        "topology": "chain",
        "lr": "small",
    },
    "RelaySum/model (with local nesterov momentum, strip, w/ lr correction)": {
        "algorithm": "RelaySum/Model",
        "momentum": 0.9,
        "momentum_type": "local",
        "topology": "chain",
        "lr": "normal",
    },
    "RelaySum/model (with local nesterov momentum, double-binary-tree, w/ lr correction)": {
        "algorithm": "RelaySum/Model",
        "momentum": 0.9,
        "momentum_type": "local",
        "topology": "double-binary-trees",
        "lr": "normal",
    },
    "centralized SGD with local momentum and with fully connected graph": {
        "algorithm": "All-reduce",
        "momentum": 0.9,
        "momentum_type": "local",
        "topology": "fully-connected",
        "lr": "normal",
    }
}

In [157]:
results = []
for key, value in pd.read_pickle("data/resnet_evonorm18_on_imagenet_with_01_alpha.pickle").items():
    props = properties[key]
    accuracy = sum(value["averaged top-1 of local models (each on full test dataset)"][-5:])/5
    results.append({
        **props,
        "metric": "accuracy",
        "value": accuracy / 100,
    })
df = pd.DataFrame(results).set_index(["algorithm", "topology", "lr", "momentum", "momentum_type", "metric"])
df

value
algorithm      topology            lr     momentum momentum_type metric            
gossip         exponential         normal 0.9      local         accuracy  0.584992
                                                   quasi-global  accuracy  0.615232
RelaySum/Model 3-tree              small  0.9      local         accuracy  0.591369
               chain               small  0.9      local         accuracy  0.586581
                                   normal 0.9      local         accuracy  0.600344
               double-binary-trees normal 0.9      local         accuracy  0.600021
gossip         ring                normal 0.9      quasi-global  accuracy  0.558244
All-reduce     fully-connected     normal 0.9      local         accuracy  0.696816

In [158]:
@register_filter
def percentage(value):
    val = value * 100
    return f"{val:.1f}\%"

def acc(algorithm, topology, lr, momentum_type):
    return df.loc[algorithm, topology, lr, 0.9, momentum_type, "accuracy"]["value"].item()

In [159]:
register_global("acc", acc)

In [160]:
%%template table
\tablefontsize
\centering
\begin{tabularx}{\textwidth}{l X l}
    \toprule
    Algorithm & Topology & Top-1 Accuracy \\
    \cmidrule(lr){1-2}     \cmidrule(lr){3-3}
    Centralized {\color{gray}(baseline)} & fully-connected & \Var{acc("All-reduce", "fully-connected", "normal", "local") | percentage} \\
    \textbf{\RelaySumModel w/ momentum} & double binary trees & \Var{acc("RelaySum/Model", "double-binary-trees", "normal", "local") | percentage} \\
    \dpsgd \cite{lian2017dpsgd} w/ quasi-global momentum \cite{lin2021quasiglobal} & ring & \Var{acc("gossip", "ring", "normal", "quasi-global") | percentage} \\
    \dsquare \cite{tang2018d2} w/ momentum & ring & {\color{gray}diverged at epoch 65, at 49.5\%} \\
    SGP \cite{assran2019sgp} w/ momentum & time-varying exponential \cite{assran2019sgp} & \Var{acc("gossip", "exponential", "normal", "local") | percentage} \\
    \bottomrule
\end{tabularx}

In [161]:
!mkdir -p generated
with open("generated/imagenet-results.tex", "w") as fp:
    res = render("table")
    fp.write(res)
    print(res)

\tablefontsize
\centering
\begin{tabularx}{\textwidth}{l X l}
    \toprule
    Algorithm & Topology & Top-1 Accuracy \\
    \cmidrule(lr){1-2}     \cmidrule(lr){3-3}
    Centralized {\color{gray}(baseline)} & fully-connected & 69.7\% \\
    \textbf{\RelaySumModel w/ momentum} & double binary trees & 60.0\% \\
    \dpsgd \cite{lian2017dpsgd} w/ quasi-global momentum \cite{lin2021quasiglobal} & ring & 55.8\% \\
    \dsquare \cite{tang2018d2} w/ momentum & ring & {\color{gray}diverged at epoch 65, at 49.5\%} \\
    SGP \cite{assran2019sgp} w/ momentum & time-varying exponential \cite{assran2019sgp} & 58.5\% \\
    \bottomrule
\end{tabularx}
